In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-12"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10163,2024-05-12,Brasil Nbb,14:00,R10 Score Vasco da Gama,Bauru,1.80,1.92,148.5,1.84,1.86,-1.5,1.93,2.09,tvjq8Cm1,0.555556,0.520833,0.543478,0.537634,0.076389,1.0,0.6,NaN,NaN,135.732,36.936197,0.272126,131.750,16.191638,0.122897,118.648,181.400,116.62,148.20,0.0,0.0,0.0,0.0,0.045620,0.007644,0.056287,2.00,0.400,2.000000,0.649161,0.8,0.150839,4.76,0.952,0.966387,0.470610,0.5,0.029390
10164,2024-05-12,Espanha Acb,13:30,Granada,Gran Canaria,1.90,1.90,166.5,1.81,1.95,-2.5,2.01,2.31,Cx5CnJs0,0.526316,0.526316,0.552486,0.512821,0.052632,0.6,0.0,NaN,NaN,227.734,98.039738,0.430501,111.334,25.901814,0.232650,166.666,147.654,182.32,122.83,0.0,0.0,0.0,0.0,0.000000,0.052657,0.098209,0.34,0.068,13.235294,0.000000,0.0,0.000000,-3.42,-0.684,-1.315789,0.474015,0.5,0.025985
10165,2024-05-12,Espanha Acb,13:30,Tenerife,Manresa,1.62,2.31,166.5,1.80,1.96,-2.5,1.90,2.48,CEMPeNKD,0.617284,0.432900,0.555556,0.510204,0.050184,0.6,0.6,NaN,NaN,156.350,17.303791,0.110673,179.098,24.486744,0.136723,144.820,168.464,165.30,145.60,0.0,0.0,0.0,0.0,0.248297,0.060179,0.187270,1.15,0.230,2.695652,0.665161,0.6,-0.065161,2.61,0.522,2.509579,0.394066,0.6,0.205934
10166,2024-05-12,Eua Nba,21:00,Minnesota Timberwolves,Denver Nuggets,1.68,2.27,203.5,1.80,1.95,-4.5,2.05,3.05,lnOqvsp4,0.595238,0.440529,0.555556,0.512821,0.035767,0.8,0.8,NaN,NaN,231.272,78.321902,0.338657,189.408,72.054265,0.380418,183.592,207.562,139.50,299.52,0.0,0.0,0.0,0.0,0.211237,0.056569,0.277297,0.20,0.040,17.000000,0.604946,0.4,-0.204946,1.03,0.206,6.165049,0.559403,0.6,0.040597
10167,2024-05-12,Alemanha Bbl,10:30,Oldenburg,Bamberg,1.61,2.33,169.5,1.80,1.96,-2.5,1.80,2.71,A58PUHCb,0.621118,0.429185,0.555556,0.510204,0.050303,0.8,0.6,NaN,NaN,189.460,83.978058,0.443250,182.176,26.163817,0.143618,145.824,264.122,135.28,184.21,0.0,0.0,0.0,0.0,0.258435,0.060179,0.285351,-0.35,-0.070,-8.714286,0.000000,0.0,0.000000,-1.21,-0.242,-5.495868,0.407336,0.3,-0.107336
10168,2024-05-12,Austrália Nbl1 Norte,01:00,South West Metro Pirates,Brisbane Capitals,1.95,1.77,181.5,1.85,1.85,-4.5,1.98,2.46,rq8rNMfb,0.512821,0.564972,0.540541,0.540541,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.068430,0.000000,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10169,2024-05-12,Áustria Superliga,12:30,UBSC Graz,Klosterneuburg,2.01,1.75,159.5,1.80,1.86,1.5,1.87,1.61,t2l6Hh0n,0.497512,0.571429,0.555556,0.537634,0.068941,0.8,1.0,NaN,NaN,161.210,7.861895,0.048768,126.080,34.557323,0.274090,173.356,153.024,175.12,164.61,0.0,0.0,0.0,0.0,0.097791,0.023184,0.105660,-1.13,-0.226,-4.469027,0.552836,0.5,-0.052836,3.52,0.704,1.065341,0.623902,0.8,0.176098
10170,2024-05-12,Áustria Superliga,13:00,Oberwart,Traiskirchen,1.66,2.17,145.5,1.87,1.87,-2.5,1.87,2.55,pOtuYRc6,0.602410,0.460829,0.534759,0.534759,0.063239,0.6,0.4,NaN,NaN,190.076,92.619936,0.487278,145.646,53.528947,0.367528,177.388,148.472,126.88,138.96,0.0,0.0,0.0,0.0,0.188316,0.000000,0.217571,-1.87,-0.374,-1.764706,0.533441,0.5,-0.033441,-2.78,-0.556,-2.104317,0.000000,0.0,0.000000
10171,2024-05-12,Bulgária Nbl,13:00,Spartak Pleven,Rilski Sportist,2.01,1.73,162.5,1.85,1.85,1.5,1.85,1.61,tbh7nZSj,0.497512,0.578035,0.540541,0.540541,0.075547,0.6,1.0,NaN,NaN,169.882,58.783244,0.346024,124.764,16.326311,0.130858,130.094,137.476,142.56,140.94,0.0,0.0,0.0,0.0,0.105877,0.000000,0.098096,2.38,0.476,2.121849,0.721588,1.0,0.278412,-0.37,-0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Brasil Nbb,R10 Score Vasco da Gama,Bauru,148.5,1.84,1.0000,Over
1,13:30,Espanha Acb,Granada,Gran Canaria,166.5,1.81,1.0000,Over
2,13:30,Espanha Acb,Tenerife,Manresa,166.5,1.80,1.0000,Over
3,21:00,Eua Nba,Minnesota Timberwolves,Denver Nuggets,203.5,1.80,1.0000,Over
4,10:30,Alemanha Bbl,Oldenburg,Bamberg,169.5,1.80,1.0000,Over
5,01:00,Austrália Nbl1 Norte,South West Metro Pirates,Brisbane Capitals,181.5,1.85,1.0000,Over
6,13:00,Áustria Superliga,Oberwart,Traiskirchen,145.5,1.87,1.0000,Over
7,04:00,Filipinas Copa Das Filipinas,Rain Or Shine Paint.,TNT Tropang Giga,209.5,1.86,1.0000,Over
8,11:20,Lituânia Lkl,Juventus,Lietkabelis,167.5,1.87,1.0000,Over
9,00:15,México Cibacopa,Ostioneros de Guaymas,Halcones de Obregon,182.5,1.85,1.0000,Over
